In [ ]:
#-------------------LOADING IN OUR OWN TRAINING & TESTING DATASET--------------------------
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

TRAIN_DATA_DIR = "cats_and_dogs_filtered\\train"
TEST_DATA_DIR = "cats_and_dogs_filtered\\validation"
CLASSES = ["dogs", "cats"]

training_data = []
testing_data = []

IMG_WIDTH = 224
IMG_HEIGHT = 224

def create_training_data():
    for the_class in CLASSES:
        path = os.path.join(TRAIN_DATA_DIR, the_class)   # Path to cats or dogs directory
        class_num = CLASSES.index(the_class)   #Giving labels to each image. 0 for Dog and 1 for Cat.
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))  # We're converting read images into grayscale image since color is not essential for this classification task
                new_array = cv2.resize(img_array, (IMG_WIDTH, IMG_HEIGHT))  #Normalizing or Resizing the dataset so that each image is of same size
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

def create_testing_data():
    for the_class in CLASSES:
        path = os.path.join(TEST_DATA_DIR, the_class)   # Path to cats or dogs directory
        class_num = CLASSES.index(the_class)   #Giving labels to each image. 0 for Dog and 1 for Cat.
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img))  # We're converting read images into grayscale image since color is not essential for this classification task
            new_array = cv2.resize(img_array, (IMG_WIDTH, IMG_HEIGHT))  #Normalizing or Resizing the dataset so that each image is of same size
            testing_data.append([new_array, class_num]) 


create_training_data()
create_testing_data()

print("Total Training Images: "+str(len(training_data)))
print("Total Testing Images: "+str(len(testing_data)))


In [ ]:
#------------------------RESHAPING THE TRAINING DATASET--------------------------
X_train = []   #Will contain training images
Y_train = []   #Will contain labels for training images

X_test = []   #Will contain testing images
Y_test = []  #Will contain labels for testing images

for image, label in training_data:
    X_train.append(image)
    Y_train.append(label)

for image, label in testing_data:
    X_test.append(image)
    Y_test.append(label)
    
X_train = np.array(X_train)
X_test = np.array(X_test)

X_train = np.array(X_train).reshape(X_train.shape[0], IMG_WIDTH, IMG_HEIGHT, 3)                                                                #-1 tells us the number of features/image. Then we have the size of image i.e width and height and last 1 tells us that the image is gray scale
X_test = np.array(X_test).reshape(X_test.shape[0], IMG_WIDTH, IMG_HEIGHT, 3)                                                                   #-1 tells us the number of features/image. Then we have the size of image i.e width and height and last 1 tells us that the image is gray scale

print(X_train.shape)
print(X_test.shape)

print("Reshaping completed!")

In [ ]:
#-------------------------SAVING THE TRAINING & TESTING DATASET--------------------------
#Saving the training features and labels
np.save('train_features.npy', X_train) 
np.save('train_labels.npy',Y_train)
print("Training data saved")

#Saving the testing features and labels
np.save('test_features.npy', X_test) 
np.save('test_labels.npy',Y_test)
print("Testing data saved")

In [ ]:
#-------------------------LOADING THE TRAINING & TESTING DATASET--------------------------
#Loading training features and labels
import numpy as np
X_train=np.load('train_features.npy')
Y_train=np.load('train_labels.npy')
print("Training data loaded")
print(X_train.shape)
print(Y_train.shape)

#Loading testing features and labels
X_test=np.load('test_features.npy')
Y_test=np.load('test_labels.npy')
print("Testing data loaded")
print(X_test.shape)
print(Y_test.shape)

In [ ]:
#-----------------------------------------------DEFINING & SAVING OUR CNN MODEL---------------------------------------------------------
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import glob
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K


old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

num_classes = 2
batch_size = 128
IMG_WIDTH = 150
IMG_HEIGHT = 150
epochs = 50


# Normalizing the data
X_train = X_train/255.0
X_test = X_test/255.0

input_shape = (IMG_WIDTH, IMG_HEIGHT, 3)

# Convert class vectors to binary class matrices
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.to_categorical(Y_test, num_classes) 


#Defining the structure of the model
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='sigmoid'))

model.add(Dense(num_classes, activation='softmax'))
          
#Adding the parameters for the model
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

#Training the model
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test))

#---------------------------------------------------SAVING THE MODEL---------------------------------------------------------
model.save('catdog_classification-CNN.model')
print("Model Saved: catdog_classification-CNN.model")

In [3]:
#-------------------------------------DATA AUGMENTATION AND THEN TRAINING---------------------------------------------------
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAIN_DATA_DIR = "cats_and_dogs_filtered\\train"
TEST_DATA_DIR = "cats_and_dogs_filtered\\validation"

num_classes = 2
batch_size = 128
IMG_WIDTH = 150
IMG_HEIGHT = 150
epochs = 50

datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Adding rescale, rotation_range, width_shift_range, height_shift_range,
# shear_range, zoom_range, and horizontal flip to our ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        TRAIN_DATA_DIR,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 32 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        TEST_DATA_DIR,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')


# Our input feature map is 150x150x3: 150x150 for the image pixels, and 3 for
# the three color channels: R, G, and B
img_input = layers.Input(shape=(150, 150, 3))

# First convolution extracts 16 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

# Second convolution extracts 32 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Third convolution extracts 64 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Convolution2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Convolution2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Flatten feature map to a 1-dim tensor
x = layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden units
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

x = layers.Dense(1024, activation='relu')(x)

# Create output layer with a single node and sigmoid activation
output = layers.Dense(1, activation='sigmoid')(x)

# Configure and compile the model
model = Model(img_input, output)
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

#history = model.fit_generator(
#      train_generator,
#      steps_per_epoch=100,
#      epochs=30,
#      validation_data=validation_generator,
#      validation_steps=50,
#      verbose=1)

#Training the model
model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=1)

#---------------------------------------------------SAVING THE MODEL---------------------------------------------------------
model.save('New-catdog_classification-CNN.model')
print("Model Saved: New-catdog_classification-CNN.model")

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/30
100/100 [==============================] - 22s 218ms/step - loss: 0.7082 - acc: 0.4995 - val_loss: 0.6913 - val_acc: 0.6490
Epoch 2/30
100/100 [==============================] - 18s 178ms/step - loss: 0.6955 - acc: 0.5125 - val_loss: 0.6713 - val_acc: 0.5630
Epoch 3/30
100/100 [==============================] - 18s 181ms/step - loss: 0.6887 - acc: 0.5675 - val_loss: 0.6983 - val_acc: 0.5110
Epoch 4/30
100/100 [==============================] - 18s 182ms/step - loss: 0.6601 - acc: 0.6030 - val_loss: 0.6426 - val_acc: 0.6580
Epoch 5/30
100/100 [==============================] - 18s 182ms/step - loss: 0.6598 - acc: 0.6210 - val_loss: 0.6002 - val_acc: 0.6850
Epoch 6/30
100/100 [==============================] - 18s 181ms/step - loss: 0.6420 - acc: 0.6470 - val_loss: 0.6107 - val_acc: 0.6520
Epoch 7/30
100/100 [==============================] - 18s 177ms/step - loss: 0.6351 - acc: 0.6555 - val_lo

In [ ]:
#-----------------------------------------------DEFINING & SAVING RESNET MODEL---------------------------------------------------------
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

num_classes = 2
batch_size = 128
IMG_WIDTH = 50
IMG_HEIGHT = 50
epochs = 10


input_shape = (IMG_WIDTH, IMG_HEIGHT, 3)


def ResNet50(input_shape=input_shape, classes=num_classes):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

model = ResNet50(input_shape = input_shape, classes = num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("Number of training examples = " + str(X_train.shape[0]))
print ("Number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

In [ ]:
#-----------------------------------------------DEFINING & SAVING VGG MODEL---------------------------------------------------------
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import glob
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2, numpy as np


old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

num_classes = 2
batch_size = 28
IMG_WIDTH = 224
IMG_HEIGHT = 224
epochs = 10


# Normalizing the data
X_train = X_train/255.0
X_test = X_test/255.0

input_shape = (IMG_WIDTH, IMG_HEIGHT, 3)

# Convert class vectors to binary class matrices
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.to_categorical(Y_test, num_classes) 

print(X_train.shape)
print(Y_train.shape)

#Defining the structure of the model
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))          
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(ZeroPadding2D((1,1)))          
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))          
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(ZeroPadding2D((1,1)))          
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))          
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))          
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(ZeroPadding2D((1,1)))          
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))          
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))          
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(ZeroPadding2D((1,1)))          
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))          
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))          
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
          
#Adding the parameters for the model
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='adam', loss=keras.losses.binary_crossentropy, metrics=['accuracy'])

#Training the model
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
         validation_data=(X_test, Y_test))

#---------------------------------------------------SAVING THE MODEL---------------------------------------------------------
model.save('catdog_classification-VGG.model')
print("Model Saved: catdog_classification-VGG.model")

In [ ]:
#---------------------------------------------------lOADING OUR CNN MODEL---------------------------------------------------------
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import glob
import keras
from numpy import array

old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

loaded_model = keras.models.load_model('catdog_classification-CNN.model')
print("Model loaded: catdog_classification-CNN.model")

In [ ]:
#---------------------------------------------------lOADING VGG MODEL---------------------------------------------------------
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import glob
import keras
from numpy import array

old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

loaded_model = keras.models.load_model('catdog_classification-VGG.model')
print("Model loaded: catdog_classification-VGG.model")

In [ ]:
#-----------------------------------------EVALUATING VALIDATION LOSS AND ACCURACY---------------------------------------------------------
val_loss, val_acc = loaded_model.evaluate(X_test, Y_test)
print("\nValidation Loss: "+str(val_loss*100)+"%")
print("Validation Accuracy: "+str(val_acc*100)+"%")

In [ ]:
#------------------------------------------PREDICTING OUTPUT OF GIVEN TEST IMAGES----------------------------------------------------
image_dir = "Test Images/"
test_images = []
IMG_WIDTH = 50
IMG_HEIGHT = 50
for filename in glob.glob(image_dir+"*.jpg"): #Searching for png images in Test Images
    img = cv2.imread(filename)
    resized_image = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT)) 
    test_images.append(resized_image)

test_images = array(test_images)   #Converting list to numpy array
test_images = test_images/255.0
test_images = test_images.reshape(test_images.shape[0], IMG_WIDTH, IMG_HEIGHT, 3)
predictions = loaded_model.predict([test_images])
for i in range(len(test_images)):
    plt.imshow(test_images[i])
    plt.show()
    if(np.argmax(predictions[i]) == 0):
        print("Prediction: Dog")
    elif(np.argmax(predictions[i]) == 1):
        print("Prediction: Cat")        

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAIN_DATA_DIR = "cats_and_dogs_filtered\\train"
TEST_DATA_DIR = "cats_and_dogs_filtered\\validation"

datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Adding rescale, rotation_range, width_shift_range, height_shift_range,
# shear_range, zoom_range, and horizontal flip to our ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        TRAIN_DATA_DIR,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 32 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        TEST_DATA_DIR,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')